In [1]:
import pandas as pd
import numpy as np

labels = ['label_1', 'label_2', 'label_3', 'label_4']
features = [f'feature_{i}' for i in range (1,769)]

Reading Datasets

In [2]:
train_df = pd.read_csv('./11/train.csv')
test_df = pd.read_csv('./11/test.csv')
valid_df = pd.read_csv('./11/valid.csv')

Data Preprocessing -
Remove NaN values, 
Data data standardization

In [5]:
from sklearn.preprocessing import RobustScaler

id = ['ID']

x_train = {}
y_train = {}

x_valid = {}
y_valid = {}

x_test = {}

for label in labels:
  tr_df = train_df[train_df['label_2'].notna()] if label == 'label_2' else train_df
  vl_df = valid_df[valid_df['label_2'].notna()] if label == 'label_2' else valid_df
  tst_df = test_df.drop('ID', axis=1)

  scaler = RobustScaler()
  x_train[label] = pd.DataFrame(scaler.fit_transform(tr_df.drop(labels, axis = 1)), columns = features)
  y_train[label] = tr_df[label]

  x_valid[label] = pd.DataFrame(scaler.transform(vl_df.drop(labels, axis = 1)), columns = features)
  y_valid[label] = vl_df[label]

  x_test[label] = pd.DataFrame(scaler.transform(tst_df), columns = features)

In [6]:
from sklearn.decomposition import PCA

x_train_pca = {}
y_train_pca = y_train

x_valid_pca = {}
y_valid_pca = y_valid

x_test_pca = {}

for label in labels:
  pca = PCA(n_components= 0.99, svd_solver='full')
  pca.fit(x_train[label])
  x_train_trf = pd.DataFrame(pca.transform(x_train[label]))
  x_valid_trf = pd.DataFrame(pca.transform(x_valid[label]))
  x_test_trf = pd.DataFrame(pca.transform(x_test[label]))
  x_train_pca[label] = x_train_trf
  x_valid_pca[label] = x_valid_trf
  x_test_pca[label] = x_test_trf
  print(label)
  print(x_train_trf.shape)
  print(x_valid_trf.shape)
  print(x_test_trf.shape)
  print('...............................................................')

label_1
(28520, 382)
(750, 382)
(744, 382)
...............................................................
label_2
(28040, 381)
(736, 381)
(744, 381)
...............................................................
label_3
(28520, 382)
(750, 382)
(744, 382)
...............................................................
label_4
(28520, 382)
(750, 382)
(744, 382)
...............................................................


KNN Classifier for all the Labels

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

def knn(x_train, y_train, x_valid, y_valid):
  for label in labels:
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(x_train[label], y_train[label])
    y_pred = knn.predict(x_valid[label])
    print(label, metrics.accuracy_score(y_valid[label], y_pred))

knn(x_train_pca, y_train_pca, x_valid_pca, y_valid_pca)

SVM Classifier for all the Labels

Label 1 - Hyper Parameter Tuning and Cross Validation 

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np


param_dist = {
    'C': np.logspace(-2, 2, 5),  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': np.logspace(-4, 0, 5)  
}


svm = SVC()

random_search = RandomizedSearchCV(
    estimator=svm, param_distributions=param_dist, scoring='accuracy', cv=5, verbose=1, n_jobs=-1, n_iter=6, random_state=42
)


random_search.fit(x_train_pca[labels[0]], y_train_pca[labels[0]])

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print(best_params)
print(best_model)

y_pred = best_model.predict(x_valid_pca[labels[0]])
accuracy = accuracy_score(y_valid_pca[labels[0]], y_pred)
print(f'Accuracy on test data: {accuracy}')

Label 1 - Best Model

In [ ]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel= 'rbf', C=100.0, gamma=0.0001)
clf.fit(x_train_pca[labels[0]], y_train_pca[labels[0]])
y_pred = clf.predict(x_valid_pca[labels[0]])
label_1 = clf.predict(x_test_pca[labels[0]])
print(labels[0], metrics.accuracy_score(y_valid_pca[labels[0]], y_pred)) #0.94

label_1 0.952


Label 2 - Hyper Parameter Tuning and Cross Validation

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

param_dist = {
    'C': np.logspace(-3, 3, 6),  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': np.logspace(-3, 3, 6)  
}


svm = SVC()

random_search = RandomizedSearchCV(
    estimator=svm, param_distributions=param_dist, scoring='accuracy', cv=5, verbose=1, n_jobs=-1, n_iter=6, random_state=42
)


random_search.fit(x_train_pca[labels[1]], y_train_pca[labels[1]])

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print(best_params)
print(best_model)

y_pred = best_model.predict(x_valid_pca[labels[1]])
accuracy = accuracy_score(y_valid_pca[labels[1]], y_pred)
print(f'Accuracy on test data: {accuracy}')

Label 2 - Best Model

In [9]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel= 'rbf', gamma= 0.001, C= 4)
clf.fit(x_train_pca[labels[1]], y_train_pca[labels[1]])
y_pred_label_2 = clf.predict(x_valid_pca[labels[1]])
label_2 = clf.predict(x_test_pca[labels[1]])
print(labels[1], metrics.accuracy_score(y_valid_pca[labels[1]], y_pred_label_2))   #0.8872

label_2 0.8872282608695652


Label 3 - Hyper Parameter Tuning and Cross Validation

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

param_dist = {
    'C': np.logspace(-2, 2, 5),  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': np.logspace(-4, 0, 5)  
}


svm = SVC()

random_search = RandomizedSearchCV(
    estimator=svm, param_distributions=param_dist, scoring='accuracy', cv=5, verbose=1, n_jobs=-1, n_iter=6, random_state=42
)


random_search.fit(x_train_pca[labels[2]], y_train_pca[labels[2]])

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print(best_params)
print(best_model)

y_pred = best_model.predict(x_valid_pca[labels[2]])
accuracy = accuracy_score(y_valid_pca[labels[2]], y_pred)
print(f'Accuracy on test data: {accuracy}')

Label 3 - Best Model

In [15]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel= 'rbf')
clf.fit(x_train_pca[labels[2]], y_train_pca[labels[2]])
y_pred_label_3 = clf.predict(x_valid_pca[labels[2]])
label_3 = clf.predict(x_test_pca[labels[2]])
print(labels[2], metrics.accuracy_score(y_valid_pca[labels[2]], y_pred_label_3)) #0.9973333333333333

label_3 0.9973333333333333


Label 4 - Hyper Parameter Tuning and Cross Validation

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

param_dist = {
    'C': np.logspace(-3, 3, 6),  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': np.logspace(-3, 3, 6)  
}


svm = SVC()

random_search = RandomizedSearchCV(
    estimator=svm, param_distributions=param_dist, scoring='accuracy', cv=5, verbose=1, n_jobs=-1, n_iter=6, random_state=42
)


random_search.fit(x_train_pca[labels[3]], y_train_pca[labels[3]])

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print(best_params)
print(best_model)

y_pred = best_model.predict(x_valid_pca[labels[3]])
accuracy = accuracy_score(y_valid_pca[labels[3]], y_pred)
print(f'Accuracy on test data: {accuracy}')

Label 4 - Best Model

In [22]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel= 'rbf', gamma= 0.001, C= 3.981071705534969)
clf.fit(x_train_pca[labels[3]], y_train_pca[labels[3]])
y_pred_label_4 = clf.predict(x_valid_pca[labels[3]])
label_4 = clf.predict(x_test_pca[labels[3]])
print(labels[3], metrics.accuracy_score(y_valid_pca[labels[3]], y_pred_label_4)) #0.94

label_4 0.94


Making Output Dataset

In [ ]:
id_values = list(range(1, 745))
output_df = pd.DataFrame({
    'ID': id_values,
    'label_1': label_1,
    'label_2': label_2,
    'label_3': label_3,
    'label_4': label_4,
})

output_df

,ID,label_1,label_2,label_3,label_4
0,1,26,26,0,2
1,2,18,18,1,8
2,3,16,16,1,6
3,4,7,7,1,6
4,5,58,58,0,6
...,...,...,...,...,...
739,740,47,56,1,6
740,741,35,35,1,2
741,742,54,53,1,6
742,743,38,38,1,12


Writing to the CSV file

In [24]:
output_df.to_csv('./11/11.csv', index=False)

NameError: name 'output_df' is not defined